# BHS 2020 project: Brain Learning Unicorn Project

*Using UKBB to apply the new tools learned at the BHS*

Author: Elise Douard

In [3]:
# Import all the libraries of interest
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from nilearn.datasets import fetch_development_fmri

%matplotlib inline

In [11]:
# Import my dataset
UKBB_gen_MRI_path = ('/home/elise/Documents/UKBB/FreeSurfer_Desikan_parcellation/Freesurfer_aparc_volume_lh_UKBB_40k_CNVs_apr2020.csv')
UKBB_gen_MRI = pandas.read_csv(UKBB_gen_MRI_path,sep=',')

## Preliminary description of the dataset

These are the raw data from which I will extract my groups of CNV carriers and controls. 

In [12]:
# Little overview of my data
UKBB_gen_MRI.head()

,eid,sex,site,age,KirovPathCNV,lh_bankssts_volume,lh_caudalanteriorcingulate_volume,lh_caudalmiddlefrontal_volume,lh_cuneus_volume,lh_entorhinal_volume,...,lh_rostralmiddlefrontal_volume,lh_superiorfrontal_volume,lh_superiorparietal_volume,lh_superiortemporal_volume,lh_supramarginal_volume,lh_frontalpole_volume,lh_temporalpole_volume,lh_transversetemporal_volume,lh_insula_volume,Total
0,1000169,Female,11026,69.825269,control,2897,1175,6272,2877,2201,...,13333,22929,14670,11387,11828,971,2590,1008,7023,1.459688e+06
1,1000442,Female,11026,70.637097,control,2128,2464,6764,2783,1713,...,14471,21541,13495,12930,9966,1122,2591,1099,6871,1.471546e+06
2,1000478,Male,11027,64.913978,control,2990,1664,8464,3859,2042,...,15640,25236,12596,13683,11392,908,2531,1431,7896,1.640301e+06
3,1000712,Male,11025,53.120968,control,3569,1070,7927,3395,1781,...,19369,25941,15435,14427,16230,983,2840,1417,7405,1.718176e+06
4,1000740,Female,11025,66.413978,control,2557,1619,6184,2570,1591,...,14193,21945,14658,12577,10687,826,2545,1275,6950,1.501684e+06


In [20]:
# Show the description of the columns with categorie variables
UKBB_gen_MRI.describe(include = 'object')

,sex,KirovPathCNV
count,35759,35759
unique,2,65
top,Female,control
freq,18951,34494


Kendall and Kirov et al. (2019) have already annotated the individuals carriers of at least on known recurrent CNV ("KirovPathCNV" column). 
**BUT** the CNVs used in their list are not all pathogenic - meaning they are not all fully penetrant/highly associated to psychiatric disorders.

In [19]:
x=34494-35759
x
UKBB_gen_MRI['KirovPathCNV'].value_counts()

control                                34494
15q13.3dup(CHRNA7)                       235
2q13del(NPHP1)                           223
2q13dup(NPHP1)                           189
15q11.2dup                               147
                                       ...  
2q13dup(NPHP1) 16p13.11dup                 1
10q11.21q11.23del 7q11.23dup_distal        1
13q12.12dup   2q13del(NPHP1)               1
17p13.3(PAFAH1B1)dup                       1
8p23.1dup                                  1
Name: KirovPathCNV, Length: 65, dtype: int64

I want to select only the ones from my list of pathogenic CNVs